<a href="https://colab.research.google.com/github/ajenningsfrankston/riiid_kt/blob/master/Riiid_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"


In [3]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle


/content/gdrive/My Drive/Kaggle


In [10]:
#!kaggle competitions download --force -c riiid-test-answer-prediction 



100% 59.0/59.0 [00:00<00:00, 92.8B/s]

100% 445k/445k [00:00<00:00, 1.92MB/s]

100% 9.48k/9.48k [00:00<00:00, 44.1kB/s]

100% 1.29G/1.29G [00:35<00:00, 87.4MB/s]
100% 1.29G/1.29G [00:35<00:00, 38.7MB/s]
100% 971/971 [00:00<00:00, 5.99kB/s]

100% 289k/289k [00:00<00:00, 1.67MB/s]

100% 5.99k/5.99k [00:00<00:00, 38.1kB/s]



In [20]:
import os

if not os.path.exists('../input/riiid-test-answer-prediction'):
    os.makedirs('../input/riiid-test-answer-prediction')

import zipfile

with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('../input/riiid-test-answer-prediction')



In [25]:
import shutil
    
source_dir = './'
target_dir = '../input/riiid-test-answer-prediction'
    
file_names = ['lectures.csv','questions.csv','example_test.csv','example_sample_submission.csv']
    
for file_name in file_names:
    if not os.path.exists(target_dir + '/' + file_name):
        shutil.move(os.path.join(source_dir, file_name), target_dir)

os.listdir('../input/riiid-test-answer-prediction')

['example_sample_submission.csv',
 'example_test.csv',
 'lectures.csv',
 'questions.csv',
 'train.csv']